In [5]:
import urllib, demjson, os, json, requests, time, random
import pandas as pd 
import numpy as np

In [6]:
# track to not overload google's API
how_far_indexed = 0
# track every image to latter append to DF
imagelist = []

In [79]:
#new york times best sellers
data = pd.read_json('nyt2.json', lines=True)

In [8]:
#information needed
author_title_best = data[['author','title']]

In [77]:
#drop duplicate best seller entries
author_title_best = author_title_best.drop_duplicates(subset=['title'])

In [10]:
#try my best to remove difficult charators
author_title_best['author'].str.encode('ascii', 'ignore').str.decode('ascii')
author_title_best['title'].str.encode('ascii', 'ignore').str.decode('ascii')

0                       ODD HOURS
1                        THE HOST
2        LOVE THE ONE YOU'RE WITH
3                       THE FRONT
4                           SNUFF
                   ...           
10190    TOM CLANCY LINE OF SIGHT
10191      SOMETHING IN THE WATER
10192     LITTLE FIRES EVERYWHERE
10193            SHELTER IN PLACE
10194        THE LAST TIME I LIED
Name: title, Length: 10195, dtype: object

In [97]:
# author_title_best
json_best = []
removelist = []
for i in range(950):
    author = author_title_best.iloc[i,0]
    title = author_title_best.iloc[i,1]
    url = f'''https://www.googleapis.com/books/v1/volumes?q=intitle:\"{title}\"+inauthor:{author}&maxResults=1'''
    url = url.replace(' ','%20')
    try:
        response = urllib.request.urlopen(url)
        temp_data = json.loads(response.read())
        json_best.append(temp_data)
        time.sleep(random.uniform(.5, 1))
    except UnicodeEncodeError:
        removelist.append(i)


yed her entire family and left her penniless. Her biggest asset is a mysterious man known only as Kere.'}}]}
{'kind': 'books#volumes', 'totalItems': 10, 'items': [{'kind': 'books#volume', 'id': 'UKYeb_dZFwsC', 'etag': 'QZm63xrVTyk', 'selfLink': 'https://www.googleapis.com/books/v1/volumes/UKYeb_dZFwsC', 'volumeInfo': {'title': 'Skinnydipping', 'subtitle': 'A Novel', 'authors': ['Bethenny Frankel'], 'publisher': 'Simon and Schuster', 'publishedDate': '2012-05-01', 'description': "A tale loosely based on the author's early adulthood follows the experiences of a struggling actress and outspoken businesswoman whose ambitions lead her through several high-profile relationships.", 'industryIdentifiers': [{'type': 'ISBN_13', 'identifier': '9781451667370'}, {'type': 'ISBN_10', 'identifier': '145166737X'}], 'readingModes': {'text': False, 'image': False}, 'pageCount': 368, 'printType': 'BOOK', 'categories': ['Fiction'], 'averageRating': 4, 'ratingsCount': 2, 'maturityRating': 'NOT_MATURE', 'all

In [18]:
#run once after every every call to remove book names that where not red
author_title_best.drop(author_title_best.index[removelist])
how_far_indexed += len(json_best)
#now you can run the above function again

In [91]:
#find the url in the json file
def gettingUrl(file):
    try:
        current = json.loads(json.dumps(file))['items']
        current = json.loads(json.dumps(current[0]))['volumeInfo']
        title = json.loads(json.dumps(current))['title']
        author = json.loads(json.dumps(current))['authors'][0]
        current = json.loads(json.dumps(current))['imageLinks']
        url = json.loads(json.dumps(current))['smallThumbnail']
        return url, title, author
    except KeyError:
        return 'fail', '', ''

In [112]:
json_best[100]

{'kind': 'books#volumes',
 'totalItems': 20,
 'items': [{'kind': 'books#volume',
   'id': 'ObWZAAAAQBAJ',
   'etag': 'A6919X/NFYw',
   'selfLink': 'https://www.googleapis.com/books/v1/volumes/ObWZAAAAQBAJ',
   'volumeInfo': {'title': 'One Fifth Avenue',
    'authors': ['Candace Bushnell'],
    'publisher': 'Hachette Books',
    'publishedDate': '2008-09-22',
    'description': 'From one of the most consistently astute and engaging social commentators of our day comes another look at the tough and tender women of New York City--this time, through the lens of where they live. One Fifth Avenue, the Art Deco beauty towering over one of Manhattan\'s oldest and most historically hip neighborhoods, is a one-of-a-kind address, the sort of building you have to earn your way into--one way or another. For the women in Candace Bushnell\'s new novel, One Fifth Avenue, this edifice is essential to the lives they\'ve carefully established--or hope to establish. From the hedge fund king\'s wife to the

In [98]:
urllist = []
for idx, jsonFile in enumerate(json_best):
    curr_url, title, author = (gettingUrl(jsonFile))
    if (curr_url == 'fail'):
        continue
    img_data = requests.get(curr_url).content
    title = title.replace('/',' ')
    with open(f'best_sellers_images/{title}.jpg', 'wb') as handler:
        handler.write(img_data)
    imagelist.append(f'{title}.jpg')
    authorlist.append(author)
    titlelist.append(title)


In [105]:
best_sellers = pd.DataFrame(list(zip(titlelist, authorlist,imagelist)),columns =['title', 'author','imagelist']) 

In [106]:
best_sellers = best_sellers.drop_duplicates(subset=['title'])
best_sellers['best'] = 1

In [109]:
best_sellers.to_csv('best_sellers.csv')